In [1]:
!pip install -q -U openai langchain-openai anthropic docling easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Version 7, ANTHROPIC

### PARSE PDF

In [ ]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
_log = logging.getLogger(__name__)
IMAGE_RESOLUTION_SCALE = 5.0
def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path("/kaggle/input/plumbing/RedHead Rooter Service Call Tree.pdf")
    output_dir = Path("imgs")

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    # Save page images
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.jpeg"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="JPEG")

    # Save images of figures and tables
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-table-{table_counter}.jpeg"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "JPEG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-picture-{picture_counter}.jpeg"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "JPEG")

    end_time = time.time() - start_time

    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")
main()

### GET NODES

In [7]:
import easyocr
reader = easyocr.Reader(['ch_sim','en'], )
nodes = reader.readtext('/kaggle/working/imgs/RedHead Rooter Service Call Tree-1.jpeg',
                        detail = 0, paragraph=True,
                        canvas_size=5000,
                        decoder="wordbeamsearch",
                        ## experiment below. Ref: https://www.jaided.ai/easyocr/documentation/
                        beamWidth = 10,
                        width_ths=0.9, 
                        text_threshold=0.6,
                        )
nodes

['RedHead Rooter Service Call Tree',
 'CallArrival',
 'RedHead Rooter! How may Ihelp yoU today?',
 'Immediately call the on-call technician and provide the Customer contact information 9d address to them_ Follow Upas needed。',
 'Customer Service (Past customer or customer with current booking)',
 'Lead',
 'Personal Call',
 'Salesperson',
 '(needs plumbing services)',
 'Angi Call',
 'Calling about UPCOMING booking-',
 'Calling about a PASTICOMPLETED booking-',
 'Collect and log Basic Information in Talkdesk Contact: Name Address Phone Number Email Address How they heard of RedHead Rooter',
 'Does not get personal calls on his business phone',
 "Politely decline and insist you're not interested",
 'Determine the reason fortheir Call',
 'Determine What the customer needs help with',
 'It is urgent / customeris upset',
 'Customer wants to book another appointment',
 'Record the issue and reach out to Shane immediately evenaftechours (be sure to include indicators of urgency)',
 'Verify cus

### LINEARIZE DIAGRAM

In [21]:
import os
os.environ["ANTHROPIC_API_KEY"] = "kkk"
os.environ['OPENAI_API_KEY'] = "kkk"
import anthropic
import base64
from textwrap import dedent

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


image_path = "/kaggle/working/imgs/RedHead Rooter Service Call Tree-1.jpeg"
base64_image = encode_image(image_path)

client = anthropic.Anthropic()

prompt = """The workflow in the provided image is a process diagram that describes steps or nodes that an customer service human agent handles phone calls from a person in different cases.
Based on the example above, your task is to interpret the diagram and generate all various relationships found in it.
Each box means a node.
### Strictly comply with the following rules:
- In the output, include all boxes and texts found in the diagram, and keep the texts as they are. Expand texts are they are in the original diagram.
- In the output, each Step corresponds with a box of any shape (rectangle, haxagon or diamond). Include all the nodes until you find no more.
- Use If-clauses in the current step to show steps that may conditionally come next.
- In the If-clause, always use "go to Step <step_id>" as the reference to the next step. 
These are extremely important requirements you must follow.

The following elements are nodes or conditions in the diagram. You have to consider them all together with the diagram when creating the output. Ensure to use all the elements in your output because they're the essential nodes and conditions.
### Elements to consider:
{nodes}

Here is an example illustrating the output template:
**Step 3: Lead**: The caller needs plumbing services.
    Step 3.1: Take these substeps in order
        1. Ask the customer back about their basic information if it's not available initially: Name, Address, Phone Number and Email Address.
        2. Next, log the information in Talkdesk Contact. 
        3. Then go to Step 3.2
    Step 3.2: Verify customer is in service range using customer Map. If customer is not in service range, go to Step 3.3, else go to Step 3.4              
    Step 3.3:
        - Log customer information in Talkdesk Contact
        - Let the customer know we are unable to service them
        - End the call
    Step 3.4: ...
""".format(nodes=nodes)
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    temperature=0.3,
    max_tokens=4024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": base64_image,
                    },
                },
                {
                    "type": "text",
                    "text": prompt
                }
            ],
        }
    ],
)

diagram_text = message.content[0].text
print(diagram_text)

I'll help create a structured workflow based on the service call tree diagram. Here's the detailed step-by-step breakdown:

**Step 1: Call Arrival**: Initial point when call comes in with "RedHead Rooter! How may I help you today?"
If Personal Call, go to Step 2
If Salesperson, go to Step 3
If Lead (needs plumbing services), go to Step 4
If Customer Service (Past customer or customer with current booking), go to Step 5

**Step 2: Personal Call**:
- Response: "Does not get personal calls on his business phone"
- End call

**Step 3: Salesperson**:
- "Politely decline and insist you're not interested"
- End call

**Step 4: Lead (needs plumbing services)**:
Step 4.1: Collect and log Basic Information in Talkdesk Contact:
- Name
- Address
- Phone Number
- Email Address
- How they heard of RedHead Rooter

Step 4.2: Verify customer is in service range using customer Map
If NOT in service range, go to Step 4.3
If IN service range, go to Step 4.4

Step 4.3: 
- Log customer information in Talkde

In [29]:
reflect_prompt = dedent("""
The following text is a way to describe the nodes and their relationships in the diagram from the provided image.
<text>
{diagram_text}
</text>

Here are a list of elements that an ideal text for the diagram should include:
<elements>
{elements}
</elements>

Your task to is meticulously comparing the text against each element of the element list to see if the text misses any elements. If it does, add those elements to the text, using the diagram as a reference.
You should return:
- All elements that were not mentioned in the text.
- A complete revised diagram text with these elements seamlessly incorporated into the appropriate sections. 
The new text must be complete and contain no notes so that when you read it, you won't know that modifications are there.
""").strip().format(diagram_text=diagram_text, elements=nodes)
reflect_message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    temperature=0.3,
    max_tokens=4024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": base64_image,
                    },
                },
                {
                    "type": "text",
                    "text": reflect_prompt
                }
            ],
        }
    ],
)

reflect = reflect_message.content[0].text
print(reflect)

After comparing the text with the elements list, I found a few missing elements:

Missing elements:
1. "Angi Call" - This appears to be a type of call but wasn't mentioned in the initial branching
2. "Great! Now What Seems to be the problem?" - This follow-up question after collecting basic information
3. "Complete any required follow-up with the customer" - This appears to be a final step in some processes
4. "Call the technician the job is scheduled for immediately after booking to let them know there is a new job scheduled for them today" - This is a specific follow-up action for same-day appointments

Here's the complete revised text incorporating all elements:

**Step 1: Call Arrival**: Initial point when call comes in with "RedHead Rooter! How may I help you today?"
If Personal Call, go to Step 2
If Salesperson, go to Step 3
If Lead (needs plumbing services) or Angi Call, go to Step 4
If Customer Service (Past customer or customer with current booking), go to Step 5

**Step 2: Pe